

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/NER_PL.ipynb)




# **Detect entities in Polish language**

## 0. Colab Setup

In [ ]:
!sudo apt-get install openjdk-8-jdk
!java -version
!pip install --ignore-installed -q pyspark==2.4.4
!pip install spark-nlp

In [ ]:
import pandas as pd
import numpy as np
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 1. Start Spark Session

In [ ]:
spark = sparknlp.start()

## 2. Select the DL model and re-run cells below.

In [ ]:
MODEL_NAME = 'wikiner_6B_100'
# MODEL_NAME = 'wikiner_6B_300'
# MODEL_NAME = 'wikiner_840B_300'

LANG = 'pl'

## 3. Some sample examples

In [ ]:
text_list = ["""William Henry Gates III (ur. 28 października 1955 r.) To amerykański magnat biznesowy, programista, inwestor i filantrop. Najbardziej znany jest jako współzałożyciel Microsoft Corporation. Podczas swojej kariery w Microsoft Gates zajmował stanowiska prezesa, dyrektora generalnego (CEO), prezesa i głównego architekta oprogramowania, będąc jednocześnie największym indywidualnym akcjonariuszem do maja 2014 r. Jest jednym z najbardziej znanych przedsiębiorców i pionierów rewolucja mikrokomputerowa lat 70. i 80. Urodzony i wychowany w Seattle w stanie Waszyngton, Gates był współzałożycielem Microsoftu z przyjacielem z dzieciństwa Paulem Allenem w 1975 roku w Albuquerque w Nowym Meksyku; stała się największą na świecie firmą produkującą oprogramowanie komputerowe. Gates prowadził firmę jako prezes i dyrektor generalny, aż do ustąpienia ze stanowiska dyrektora generalnego w styczniu 2000 r., Ale pozostał przewodniczącym i został głównym architektem oprogramowania. Pod koniec lat 90. Gates był krytykowany za taktykę biznesową, którą uważano za antykonkurencyjną. Opinię tę podtrzymują liczne orzeczenia sądowe. W czerwcu 2006 r. Gates ogłosił, że przejdzie do pracy w niepełnym wymiarze godzin w Microsoft i pracy w pełnym wymiarze godzin w Bill & Melinda Gates Foundation, prywatnej fundacji charytatywnej, którą on i jego żona Melinda Gates utworzyli w 2000 r. [ 9] Stopniowo przeniósł obowiązki na Raya Ozziego i Craiga Mundie. Zrezygnował z funkcji prezesa Microsoftu w lutym 2014 r. I objął nowe stanowisko jako doradca ds. Technologii, aby wesprzeć nowo mianowaną CEO Satyę Nadellę.""",
             """Mona Lisa to XVI-wieczny obraz olejny stworzony przez Leonarda. Odbywa się w Luwrze w Paryżu.""",
]

## 4. Define Spark NLP pipeline

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Select the appropriate WordEmbeddingsModel to match the NerDLModel.
embeddings = None
if MODEL_NAME == 'wikiner_6B_100':
    embeddings = WordEmbeddingsModel.pretrained(name='glove_100d') \
        .setInputCols(["document", 'token']) \
        .setOutputCol("embeddings")
elif MODEL_NAME=='wikiner_6B_300':
    embeddings = WordEmbeddingsModel.pretrained(name='glove_6B_300', lang='xx') \
        .setInputCols(["document", 'token']) \
        .setOutputCol("embeddings")
elif MODEL_NAME=='wikiner_840B_300':
    embeddings = WordEmbeddingsModel.pretrained(name='glove_840B_300', lang='xx') \
        .setInputCols(["document", 'token']) \
        .setOutputCol("embeddings")

public_ner = NerDLModel.pretrained(MODEL_NAME, lang=LANG) \
    .setInputCols(["document", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[documentAssembler, 
                               tokenizer,
                               embeddings,
                               public_ner,
                               ner_converter])



## 5. Run the pipeline

In [ ]:
empty_df = spark.createDataFrame([['']]).toDF("text")
pipelineModel = nlpPipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
result = pipelineModel.transform(df)

## 6. Visualize Results

In [ ]:

result.select(F.explode(
    F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')) \
        .alias('cols')).select(F.expr("cols['0']").alias('chunk'),
    F.expr("cols['1']['entity']").alias('ner_label')).show(truncate=False)
result = result.toPandas()